In [27]:
import redis
import time

In [34]:
pool = redis.ConnectionPool(host="localhost", port=6379, db=0)
r = redis.Redis(connection_pool=pool)
p = r.pubsub()

p.subscribe("Queue")

p.get_message()
time.sleep(0.5)
for mes in p.listen() :
    print(mes)

{'type': 'message', 'pattern': None, 'channel': b'Queue', 'data': b'Hello World!'}
{'type': 'message', 'pattern': None, 'channel': b'Queue', 'data': b'Hello World!'}
{'type': 'message', 'pattern': None, 'channel': b'Queue', 'data': b'Hello World!'}
{'type': 'message', 'pattern': None, 'channel': b'Queue', 'data': b'Hello'}


KeyboardInterrupt: 